In [29]:
import os
import math
import numpy as np
import pandas as pd

In [30]:
data_path = os.path.join("..", "data", "demand_history.csv")
df = pd.read_csv(data_path)

df["date"] = pd.to_datetime(df["date"])
df.head()

,date,sku,units_sold
0,2024-01-01,USB_C_CABLE,34
1,2024-01-02,USB_C_CABLE,42
2,2024-01-03,USB_C_CABLE,56
3,2024-01-04,USB_C_CABLE,41
4,2024-01-05,USB_C_CABLE,54


In [31]:
daily_demand = (
    df.groupby("date", as_index=False)
      .agg(units_sold=("units_sold", "sum"))
)

daily_demand.head()

,date,units_sold
0,2024-01-01,185
1,2024-01-02,187
2,2024-01-03,211
3,2024-01-04,177
4,2024-01-05,191


In [33]:
avg_daily_demand = daily_demand["units_sold"].mean()
std_daily_demand = daily_demand["units_sold"].std()

avg_daily_demand, std_daily_demand

(231.96174863387978, 35.344235331670106)

In [34]:
lead_time_days = 7
service_level = 0.95

In [35]:
from scipy.stats import norm

z_score = norm.ppf(service_level)
z_score

1.6448536269514722

In [36]:
safety_stock_wh = z_score * std_daily_demand * math.sqrt(lead_time_days)
safety_stock_wh

153.81364606642455

In [38]:
reorder_point_wh = (avg_daily_demand * lead_time_days) + safety_stock_wh
reorder_point_wh

1777.545886503583

In [39]:
inventory_summary_wh = {
    "Average Daily Demand": round(avg_daily_demand, 2),
    "Demand Std Dev": round(std_daily_demand, 2),
    "Lead Time (days)": lead_time_days,
    "Service Level": service_level,
    "Safety Stock (units)": round(safety_stock_wh, 0),
    "Reorder Point (units)": round(reorder_point_wh, 0),
}

inventory_summary_wh

{'Average Daily Demand': 231.96,
 'Demand Std Dev': 35.34,
 'Lead Time (days)': 7,
 'Service Level': 0.95,
 'Safety Stock (units)': 154.0,
 'Reorder Point (units)': 1778.0}

In [40]:
daily_sku = (
    df.groupby(["sku", "date"], as_index=False)
      .agg(units_sold=("units_sold", "sum"))
)

daily_sku.head()

,sku,date,units_sold
0,HEADPHONES,2024-01-01,30
1,HEADPHONES,2024-01-02,43
2,HEADPHONES,2024-01-03,27
3,HEADPHONES,2024-01-04,36
4,HEADPHONES,2024-01-05,30


In [41]:
sku_stats = (
    daily_sku.groupby("sku", as_index=False)
    .agg(
        avg_daily_demand=("units_sold", "mean"),
        std_daily_demand=("units_sold", "std"),
        days_observed=("units_sold", "count")
    )
)

sku_stats["std_daily_demand"] = sku_stats["std_daily_demand"].fillna(0)
sku_stats

,sku,avg_daily_demand,std_daily_demand,days_observed
0,HEADPHONES,45.363388,9.390270,366
1,LAPTOP_STAND,42.161202,8.275312,366
2,PHONE_CHARGER,32.363388,6.658123,366
3,USB_C_CABLE,55.169399,10.555204,366
4,WIRELESS_MOUSE,56.904372,11.856869,366


In [42]:
sku_plan = sku_stats.copy()

sku_plan["mean_demand_lead_time"] = sku_plan["avg_daily_demand"] * lead_time_days
sku_plan["std_demand_lead_time"] = sku_plan["std_daily_demand"] * math.sqrt(lead_time_days)

sku_plan["safety_stock_units"] = z_score * sku_plan["std_demand_lead_time"]
sku_plan["reorder_point_units"] = (
    sku_plan["mean_demand_lead_time"] + sku_plan["safety_stock_units"]
)

sku_plan["safety_stock_units"] = sku_plan["safety_stock_units"].round().astype(int)
sku_plan["reorder_point_units"] = sku_plan["reorder_point_units"].round().astype(int)

sku_plan

,sku,avg_daily_demand,std_daily_demand,days_observed,mean_demand_lead_time,std_demand_lead_time,safety_stock_units,reorder_point_units
0,HEADPHONES,45.363388,9.390270,366,317.543716,24.844320,41,358
1,LAPTOP_STAND,42.161202,8.275312,366,295.128415,21.894418,36,331
2,PHONE_CHARGER,32.363388,6.658123,366,226.543716,17.615737,29,256
3,USB_C_CABLE,55.169399,10.555204,366,386.185792,27.926444,46,432
4,WIRELESS_MOUSE,56.904372,11.856869,366,398.330601,31.370328,52,450


In [43]:
rng = np.random.default_rng(2025)

sku_plan["on_hand_units"] = rng.integers(
    low=0,
    high=np.maximum(1, sku_plan["reorder_point_units"] * 2)
)

sku_plan["on_order_units"] = rng.integers(
    low=0,
    high=np.maximum(1, sku_plan["reorder_point_units"] // 2)
)

sku_plan["inventory_position_units"] = (
    sku_plan["on_hand_units"] + sku_plan["on_order_units"]
)

sku_plan["reorder_now_flag"] = (
    sku_plan["inventory_position_units"] <= sku_plan["reorder_point_units"]
)

sku_plan["recommended_order_qty"] = np.where(
    sku_plan["reorder_now_flag"],
    np.maximum(
        0,
        sku_plan["reorder_point_units"] * 2 - sku_plan["inventory_position_units"]
    ),
    0
).astype(int)

sku_plan.sort_values("recommended_order_qty", ascending=False)

,sku,avg_daily_demand,std_daily_demand,days_observed,mean_demand_lead_time,std_demand_lead_time,safety_stock_units,reorder_point_units,on_hand_units,on_order_units,inventory_position_units,reorder_now_flag,recommended_order_qty
0,HEADPHONES,45.363388,9.390270,366,317.543716,24.844320,41,358,320,148,468,False,0
1,LAPTOP_STAND,42.161202,8.275312,366,295.128415,21.894418,36,331,658,105,763,False,0
2,PHONE_CHARGER,32.363388,6.658123,366,226.543716,17.615737,29,256,508,107,615,False,0
3,USB_C_CABLE,55.169399,10.555204,366,386.185792,27.926444,46,432,330,165,495,False,0
4,WIRELESS_MOUSE,56.904372,11.856869,366,398.330601,31.370328,52,450,858,219,1077,False,0


In [44]:
out_path = os.path.join("..", "data", "inventory_reorder_plan.csv")
sku_plan.to_csv(out_path, index=False)

out_path

'../data/inventory_reorder_plan.csv'